# CUDA tensor network contraction demo

## Requirements
- The system must have a CUDA GPU available.

In [1]:
using Tenet
using EinExprs
using Adapt
using CUDA
using BenchmarkTools

┌ Warning: You are using a non-official build of Julia. This may cause issues with CUDA.jl.
│ Please consider using an official build from https://julialang.org/downloads/.
└ @ CUDA /home/bsc/bsc021386/.julia/packages/CUDA/75aiI/src/initialization.jl:180


Create a random tensor network and find its contraction path:

In [2]:
# Initialize random tensor network
regularity = 6
ntensors = 10
tn = rand(TensorNetwork, ntensors, regularity)
path = einexpr(tn; optimizer=Exhaustive())

SizedEinExpr{Symbol}(EinExpr{Symbol}(Symbol[], EinExpr{Symbol}[EinExpr{Symbol}([:P, :A, :c, :Y], EinExpr{Symbol}[]), EinExpr{Symbol}([:P, :A, :c, :Y], EinExpr{Symbol}[EinExpr{Symbol}([:H, :U, :F, :Z, :X, :O, :A, :c], EinExpr{Symbol}[]), EinExpr{Symbol}([:H, :P, :U, :F, :Z, :X, :O, :Y], EinExpr{Symbol}[EinExpr{Symbol}([:E, :K, :U, :V, :I, :a, :F, :Z, :b], EinExpr{Symbol}[]), EinExpr{Symbol}([:E, :H, :P, :K, :V, :I, :a, :b, :X, :O, :Y], EinExpr{Symbol}[EinExpr{Symbol}([:D, :E, :H, :M, :P], EinExpr{Symbol}[]), EinExpr{Symbol}([:D, :M, :K, :V, :I, :a, :b, :X, :O, :Y], EinExpr{Symbol}[EinExpr{Symbol}([:D, :J, :G, :a, :O, :T, :Y], EinExpr{Symbol}[EinExpr{Symbol}([:D, :J, :R], EinExpr{Symbol}[]), EinExpr{Symbol}([:R, :G, :a, :O, :T, :Y], EinExpr{Symbol}[])]), EinExpr{Symbol}([:J, :M, :G, :K, :V, :I, :b, :X, :T], EinExpr{Symbol}[EinExpr{Symbol}([:M, :G, :S, :K, :Q], EinExpr{Symbol}[]), EinExpr{Symbol}([:J, :S, :Q, :V, :I, :b, :X, :T], EinExpr{Symbol}[EinExpr{Symbol}([:V, :b, :N, :C, :L, :X, :B

Transform the tensors' data types to `CuArray`s:

In [3]:
cudatn = adapt(CuArray, tn)

TensorNetwork (#tensors=10, #inds=30)

Benchmark CUDA tensor network contraction:

In [4]:
@benchmark contract(cudatn; path)

BenchmarkTools.Trial: 1355 samples with 1 evaluation.
 Range (min … max):  854.907 μs …   9.324 ms  ┊ GC (min … max): 0.00% … 36.81%
 Time  (median):       3.749 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):     3.673 ms ± 796.963 μs  ┊ GC (mean ± σ):  0.85% ±  3.09%

  ▂                           █                                  
  █▃▁▃▁▃▃▁▁▁▁▁▁▁▃▁▁▁▁▃▃▄▃▇▆▁▁▆█▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▇ ▇
  855 μs        Histogram: log(frequency) by time        7.1 ms <

 Memory estimate: 251.72 KiB, allocs estimate: 4344.

Benchmark regular tensor network contraction:

In [5]:
@benchmark contract(tn; path)

BenchmarkTools.Trial: 12 samples with 1 evaluation.
 Range (min … max):  325.174 ms … 966.103 ms  ┊ GC (min … max):  1.81% … 57.31%
 Time  (median):     334.514 ms               ┊ GC (median):     1.78%
 Time  (mean ± σ):   433.969 ms ± 194.491 ms  ┊ GC (mean ± σ):  19.64% ± 19.62%

  █                                                              
  █▄▁▁▁▁▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄ ▁
  325 ms           Histogram: frequency by time          966 ms <

 Memory estimate: 926.79 MiB, allocs estimate: 2786.